In [64]:
#coding:utf-8
import pandas as pd
import time

from sklearn.metrics import log_loss
# instance_id 样本编号
# item_id 广告商品编号
# item_category_list 广告商品的的类目列表 分割; item_property_list_0 item_property_list_1 item_property_list_2
# item_property_list 广告商品的属性列表 分割 1 2 3
# item_brand_id 广告商品的品牌编号
# item_city_id 广告商品的城市编号
# item_price_level 广告商品的价格等级
# item_sales_level 广告商品的销量等级
# item_collected_level 广告商品被收藏次数的等级
# item_pv_level 广告商品被展示次数的等级
# user_id 用户的编号
# 'user_gender_id', 用户的预测性别编号
# 'user_age_level', 用户的预测年龄等级
# 'user_occupation_id', 用户的预测职业编号
# 'user_star_level' 用户的星级编号
# context_id 上下文信息的编号
#  context_timestamp 广告商品的展示时间
# context_page_id 广告商品的展示页面编号
# predict_category_property
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_: 
    :return: 
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

def pre_process(data):
    '''
    :param data: 
    :return: 
    '''

    print('预处理')
    print('item_category_list_ing')
    for i in range(3):
        data['category_%d'%(i)] = data['item_category_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_category_list']

    print('item_property_list_ing')
    for i in range(3):
        data['property_%d'%(i)] = data['item_property_list'].apply(
            lambda x:x.split(";")[i] if len(x.split(";")) > i else " "
        )
    del data['item_property_list']

    print('context_timestamp_ing')
    data['context_timestamp'] = data['context_timestamp'].apply(time2cov)

    print('time')
    data['context_timestamp_tmp'] = pd.to_datetime(data['context_timestamp'])
    data['week'] = data['context_timestamp_tmp'].dt.weekday
    data['hour'] = data['context_timestamp_tmp'].dt.hour
    del data['context_timestamp_tmp']


    print('predict_category_property_ing_0')
    for i in range(3):
        data['predict_category_%d'%(i)] = data['predict_category_property'].apply(
            lambda x:str(x.split(";")[i]).split(":")[0] if len(x.split(";")) > i else " "
        )

    # print('predict_category_property_ing_1')
    # for i in range(3):
    #     data['predict_property_%d'%(i)] = data['predict_category_property'].apply(
    #         lambda x:str(x.split(";")[i]).split(":")[1] if len(x.split(";")) > i else " "
    #     )
    #
    #     for j in range(3):
    #         data['predict_property_%d_%d' % (i,j)] = data['predict_property_%d'%(i)].apply(
    #             lambda x: x.split(",")[j] if len(x.split(",")) > j else -1
    #         )

    del data['predict_category_property']
    # del data['predict_property_1']
    # del data['predict_property_2']

    return data

In [65]:
print('train')
train = pd.read_csv('train.txt',sep=" ")
data2 = pd.read_csv('1234.csv',index_col=False) 
datac = pd.read_csv('clickpv.csv',index_col=False) 
train.insert(1, 'item_category_list_num', data2[['item_category_list']])
train.insert(2, 'item_property_list_num', data2[['item_property_list']])
train.insert(1, 'clickpv', datac[[ 'clickpv']])
train = pre_process(train)

all_data = train.copy()


print('all_shape',train.shape)
print(train['context_timestamp'].max())
val = train[train['context_timestamp']>'2018-09-22 23:59:59']

# 空去 22 号

train = train[train['context_timestamp']<='2018-09-22 23:59:59']

train = train[train['context_timestamp']>'2018-09-19 23:59:59']
print(train.shape)
print(val.shape)



train
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
predict_category_property_ing_0
all_shape (478138, 38)
2018-09-24 23:59:47
(207904, 38)
(121035, 38)


In [66]:
train

,instance_id,clickpv,item_category_list_num,item_property_list_num,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,...,category_1,category_2,property_0,property_1,property_2,week,hour,predict_category_0,predict_category_1,predict_category_2
78268,4293715837363757697,0.01,2,2,3412720377098676069,1975590437749032870,3948283326616421003,3,4,4,...,5799347067982556520,,2072967855524022579,5131280576272319091,2636395404473730413,4,1,5799347067982556520,7258015885215914736,7492960463130085436
78269,70764105903083828,0.01,2,1,285660928590172217,9057103201734987852,548352491538518780,8,9,8,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,21,8277336076276184272,5799347067982556520,509660095530134768
78270,3954908518456917085,0.01,2,0,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,...,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413,4,22,5755694407684602296,836752724084922533,7908382889764677758
78271,4521314384240415059,0.01,2,1,285660928590172217,9057103201734987852,548352491538518780,8,9,8,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,13,8277336076276184272,5799347067982556520,509660095530134768
78272,6005676298575614673,0.01,2,1,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,...,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413,4,9,5755694407684602296,7447522129235776380,5799347067982556520
78273,655162606126973882,0.01,2,3,285660928590172217,9057103201734987852,548352491538518780,8,9,8,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,18,2011981573061447208,8277336076276184272,5755694407684602296
78274,6388321779120371962,0.01,1,0,285660928590172217,9057103201734987852,548352491538518780,8,9,8,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,23,5755694407684602296,8710739180200009128,509660095530134768
78275,2909190494355726720,0.01,2,0,5202355029344881809,5520678735822176314,548352491538518780,8,9,10,...,5755694407684602296,,2072967855524022579,5131280576272319091,2636395404473730413,4,2,5799347067982556520,5755694407684602296,509660095530134768
78276,8631039197689382476,0.01,2,3,285660928590172217,9057103201734987852,548352491538518780,8,9,8,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,21,2011981573061447208,8277336076276184272,5755694407684602296
78277,1363985136028256226,0.01,2,3,285660928590172217,9057103201734987852,548352491538518780,8,9,8,...,8277336076276184272,,2072967855524022579,5131280576272319091,2636395404473730413,4,23,2011981573061447208,8277336076276184272,5755694407684602296


In [67]:
print('test')
test_a = pd.read_csv('test.txt',sep=" ")
data3 = pd.read_csv('456.csv',index_col=False)
data2c = pd.read_csv('test_clickpv.csv',index_col=False) 
# data2.pop('item_category_list')
test_a.insert(1, 'item_category_list_num', data3[['item_category_list']])
# data2.pop('item_property_list')
test_a.insert(2, 'item_property_list_num', data3[['item_property_list']])
test_a.insert(1, 'clickpv', data2c[[ 'clickpv']])
print(test_a.shape)
test_a = pre_process(test_a)

test
(18371, 29)
预处理
item_category_list_ing
item_property_list_ing
context_timestamp_ing
time
predict_category_property_ing_0


In [68]:
test_a

,instance_id,clickpv,item_category_list_num,item_property_list_num,item_id,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,...,category_1,category_2,property_0,property_1,property_2,week,hour,predict_category_0,predict_category_1,predict_category_2
0,2475218615076601065,0.01,2,1,2275895163219263378,7838285046767229711,7534238860363577544,7,6,8,...,8277336076276184272,,2636395404473730413,7515802706813700848,302719828435032992,1,22,8277336076276184272,5799347067982556520,509660095530134768
1,398316874173557226,0.01,2,0,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,16,5755694407684602296,8710739180200009128,7908382889764677758
2,6586402638209028583,0.01,2,2,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,14,5755694407684602296,8710739180200009128,7908382889764677758
3,1040996105851528465,0.01,2,1,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,15,8710739180200009128,5755694407684602296,7908382889764677758
4,6316278569655873454,0.01,2,4,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,15,5755694407684602296,8710739180200009128,2648343924459474923
5,868158305045921978,0.01,2,0,7096238490711246967,8126195666233054089,3948283326616421003,6,9,8,...,5755694407684602296,,5131280576272319091,2636395404473730413,7344985833148694227,1,16,8710739180200009128,5755694407684602296,509660095530134768
6,5713520501786699854,0.01,2,0,1171140458228333440,1547983184598012595,8072963182326625214,5,6,6,...,4879721024980945592,,5131280576272319091,2636395404473730413,3163265386149801264,1,11,4879721024980945592,5799347067982556520,7908382889764677758
7,932945015407923184,0.01,2,1,7020520887593189887,8324857885829459863,1019055478500227370,7,10,11,...,2011981573061447208,,5131280576272319091,1013419034389836991,5195139481388729954,1,13,2648343924459474923,2011981573061447208,5119332056286574581
8,1919197847086752313,0.01,2,0,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,...,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,1,11,2011981573061447208,7908382889764677758,8277336076276184272
9,304887065966615346,0.01,2,1,8143258757857432437,8324857885829459863,1019055478500227370,8,10,13,...,2011981573061447208,,2636395404473730413,4621934203383159480,1243768796732904674,1,14,2011981573061447208,8277336076276184272,5755694407684602296


In [69]:
# 这里是增加的内容
import datetime
def get_count_feat(all_data,data,long=3):
    end_time = data['context_timestamp'].min()
    begin_time = pd.to_datetime(end_time) - datetime.timedelta(days=long)
    all_data['context_timestamp'] = pd.to_datetime(all_data['context_timestamp'])
    all_data = all_data[
        (all_data['context_timestamp']<end_time)&(all_data['context_timestamp']>=begin_time)
                    ]
    print(end_time)
    print(begin_time)
    print(all_data['context_timestamp'].max()-all_data['context_timestamp'].min())
    item_count = all_data.groupby(['item_id'],as_index=False).size().reset_index()
    item_count.rename(columns={0:'item_count'},inplace=True)

    user_count = all_data.groupby(['user_id'], as_index=False).size().reset_index()
    user_count.rename(columns={0: 'user_count'}, inplace=True)
    return user_count,item_count


In [70]:

train_user_count,train_item_count = get_count_feat(all_data,train,2)

test_user_count,test_item_count = get_count_feat(all_data,test_a,2)

val_user_count,val_item_count = get_count_feat(all_data,val,2)

train = pd.merge(train,train_user_count,on=['user_id'],how='left')
train = pd.merge(train,train_item_count,on=['item_id'],how='left')
train = train.fillna(-1)
val = pd.merge(val,val_user_count,on=['user_id'],how='left')
val = pd.merge(val,val_item_count,on=['item_id'],how='left')
val = val.fillna(-1)
test_a = pd.merge(test_a,test_user_count,on=['user_id'],how='left')
test_a = pd.merge(test_a,test_item_count,on=['item_id'],how='left')
test_a = test_a.fillna(-1)
y_train = train.pop('is_trade')
train_index = train.pop('instance_id')

y_val = val.pop('is_trade')
val_index = val.pop('instance_id')
test_index = test_a.pop('instance_id')



2018-09-20 00:00:01
2018-09-18 00:00:01
1 days 23:59:54
2018-09-25 00:00:02
2018-09-23 00:00:02
1 days 23:59:44
2018-09-23 00:00:03
2018-09-21 00:00:03
1 days 23:59:43


In [71]:
print(test_a.shape)
del train['context_timestamp']
del val['context_timestamp']
del test_a['context_timestamp']
del all_data



(18371, 38)


In [72]:
print('baseline ing ... ...')
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from scipy import sparse
from sklearn.linear_model import LogisticRegression
print(test_a.columns)

enc = OneHotEncoder()
lb = LabelEncoder()
feat_set = list(test_a.columns)
for i,feat in enumerate(feat_set):
    tmp = lb.fit_transform((list(train[feat])+list(val[feat])+list(test_a[feat])))
    print(tmp)
    enc.fit(tmp.reshape(-1,1))
    x_train = enc.transform(lb.transform(train[feat]).reshape(-1, 1))
    x_test = enc.transform(lb.transform(test_a[feat]).reshape(-1, 1))
    x_val = enc.transform(lb.transform(val[feat]).reshape(-1, 1))
    if i == 0:
        X_train, X_test,X_val = x_train, x_test,x_val
    else:
        X_train, X_test,X_val = sparse.hstack((X_train, x_train)), sparse.hstack((X_test, x_test)),sparse.hstack((X_val, x_val))


lr = LogisticRegression()


lr.fit(X_train, y_train)
proba_val = lr.predict_proba(X_val)[:,1]
proba_sub = lr.predict_proba(X_test)[:,1]


baseline ing ... ...
Index(['clickpv', 'item_category_list_num', 'item_property_list_num',
       'item_id', 'item_brand_id', 'item_city_id', 'item_price_level',
       'item_sales_level', 'item_collected_level', 'item_pv_level', 'user_id',
       'user_gender_id', 'user_age_level', 'user_occupation_id',
       'user_star_level', 'context_id', 'context_page_id', 'shop_id',
       'shop_review_num_level', 'shop_review_positive_rate', 'shop_star_level',
       'shop_score_service', 'shop_score_delivery', 'shop_score_description',
       'category_0', 'category_1', 'category_2', 'property_0', 'property_1',
       'property_2', 'week', 'hour', 'predict_category_0',
       'predict_category_1', 'predict_category_2', 'user_count', 'item_count'],
      dtype='object')
[0 0 0 ..., 0 3 0]
[2 2 2 ..., 2 2 2]
[2 1 0 ..., 0 0 1]
[3411  309 5195 ..., 1855 1855 1855]
[ 418 1895 1182 ..., 1058 1058 1058]
[50  9  9 ..., 50 50 50]
[3 8 8 ..., 6 6 6]
[ 4  9  9 ..., 10 10 10]
[ 4  8 10 ..., 12 12 12]
[14

In [73]:
X_train.shape

(207904, 566141)

In [74]:

import lightgbm as lgb

gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=64,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.75,
                        reg_alpha = 0.4
                       )
gbm.fit(X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='binary_logloss',
        early_stopping_rounds=150)

[1]	valid_0's binary_logloss: 0.683884
Training until validation scores don't improve for 150 rounds.
[2]	valid_0's binary_logloss: 0.674812
[3]	valid_0's binary_logloss: 0.665919
[4]	valid_0's binary_logloss: 0.657191
[5]	valid_0's binary_logloss: 0.648626
[6]	valid_0's binary_logloss: 0.640232
[7]	valid_0's binary_logloss: 0.631997
[8]	valid_0's binary_logloss: 0.623911
[9]	valid_0's binary_logloss: 0.61598
[10]	valid_0's binary_logloss: 0.608196
[11]	valid_0's binary_logloss: 0.600555
[12]	valid_0's binary_logloss: 0.593051
[13]	valid_0's binary_logloss: 0.585678
[14]	valid_0's binary_logloss: 0.578442
[15]	valid_0's binary_logloss: 0.571336
[16]	valid_0's binary_logloss: 0.56436
[17]	valid_0's binary_logloss: 0.557506
[18]	valid_0's binary_logloss: 0.550768
[19]	valid_0's binary_logloss: 0.544138
[20]	valid_0's binary_logloss: 0.53763
[21]	valid_0's binary_logloss: 0.531238
[22]	valid_0's binary_logloss: 0.524947
[23]	valid_0's binary_logloss: 0.518764
[24]	valid_0's binary_logloss

[204]	valid_0's binary_logloss: 0.124553
[205]	valid_0's binary_logloss: 0.124017
[206]	valid_0's binary_logloss: 0.123485
[207]	valid_0's binary_logloss: 0.12296
[208]	valid_0's binary_logloss: 0.12244
[209]	valid_0's binary_logloss: 0.121932
[210]	valid_0's binary_logloss: 0.121422
[211]	valid_0's binary_logloss: 0.12092
[212]	valid_0's binary_logloss: 0.120424
[213]	valid_0's binary_logloss: 0.119937
[214]	valid_0's binary_logloss: 0.119452
[215]	valid_0's binary_logloss: 0.118976
[216]	valid_0's binary_logloss: 0.118506
[217]	valid_0's binary_logloss: 0.118044
[218]	valid_0's binary_logloss: 0.117583
[219]	valid_0's binary_logloss: 0.117128
[220]	valid_0's binary_logloss: 0.116681
[221]	valid_0's binary_logloss: 0.116238
[222]	valid_0's binary_logloss: 0.115803
[223]	valid_0's binary_logloss: 0.115371
[224]	valid_0's binary_logloss: 0.11495
[225]	valid_0's binary_logloss: 0.114528
[226]	valid_0's binary_logloss: 0.114114
[227]	valid_0's binary_logloss: 0.113707
[228]	valid_0's bina

[402]	valid_0's binary_logloss: 0.0860571
[403]	valid_0's binary_logloss: 0.0860204
[404]	valid_0's binary_logloss: 0.0859849
[405]	valid_0's binary_logloss: 0.0859465
[406]	valid_0's binary_logloss: 0.0859127
[407]	valid_0's binary_logloss: 0.0858794
[408]	valid_0's binary_logloss: 0.0858463
[409]	valid_0's binary_logloss: 0.0858136
[410]	valid_0's binary_logloss: 0.0857803
[411]	valid_0's binary_logloss: 0.0857474
[412]	valid_0's binary_logloss: 0.0857151
[413]	valid_0's binary_logloss: 0.0856812
[414]	valid_0's binary_logloss: 0.0856492
[415]	valid_0's binary_logloss: 0.0856204
[416]	valid_0's binary_logloss: 0.0855893
[417]	valid_0's binary_logloss: 0.0855606
[418]	valid_0's binary_logloss: 0.0855277
[419]	valid_0's binary_logloss: 0.0854972
[420]	valid_0's binary_logloss: 0.0854677
[421]	valid_0's binary_logloss: 0.0854391
[422]	valid_0's binary_logloss: 0.0854135
[423]	valid_0's binary_logloss: 0.0853838
[424]	valid_0's binary_logloss: 0.0853561
[425]	valid_0's binary_logloss: 0.

[603]	valid_0's binary_logloss: 0.0835745
[604]	valid_0's binary_logloss: 0.0835709
[605]	valid_0's binary_logloss: 0.0835657
[606]	valid_0's binary_logloss: 0.083564
[607]	valid_0's binary_logloss: 0.0835601
[608]	valid_0's binary_logloss: 0.0835565
[609]	valid_0's binary_logloss: 0.0835566
[610]	valid_0's binary_logloss: 0.0835544
[611]	valid_0's binary_logloss: 0.0835519
[612]	valid_0's binary_logloss: 0.0835496
[613]	valid_0's binary_logloss: 0.0835479
[614]	valid_0's binary_logloss: 0.0835449
[615]	valid_0's binary_logloss: 0.0835424
[616]	valid_0's binary_logloss: 0.0835399
[617]	valid_0's binary_logloss: 0.0835359
[618]	valid_0's binary_logloss: 0.0835327
[619]	valid_0's binary_logloss: 0.0835295
[620]	valid_0's binary_logloss: 0.0835254
[621]	valid_0's binary_logloss: 0.0835231
[622]	valid_0's binary_logloss: 0.0835213
[623]	valid_0's binary_logloss: 0.0835185
[624]	valid_0's binary_logloss: 0.0835163
[625]	valid_0's binary_logloss: 0.0835151
[626]	valid_0's binary_logloss: 0.0

[801]	valid_0's binary_logloss: 0.0832594
[802]	valid_0's binary_logloss: 0.0832592
[803]	valid_0's binary_logloss: 0.0832576
[804]	valid_0's binary_logloss: 0.0832581
[805]	valid_0's binary_logloss: 0.0832569
[806]	valid_0's binary_logloss: 0.0832541
[807]	valid_0's binary_logloss: 0.0832553
[808]	valid_0's binary_logloss: 0.0832556
[809]	valid_0's binary_logloss: 0.0832545
[810]	valid_0's binary_logloss: 0.0832534
[811]	valid_0's binary_logloss: 0.083253
[812]	valid_0's binary_logloss: 0.0832517
[813]	valid_0's binary_logloss: 0.083251
[814]	valid_0's binary_logloss: 0.083249
[815]	valid_0's binary_logloss: 0.083249
[816]	valid_0's binary_logloss: 0.0832482
[817]	valid_0's binary_logloss: 0.0832491
[818]	valid_0's binary_logloss: 0.0832484
[819]	valid_0's binary_logloss: 0.083248
[820]	valid_0's binary_logloss: 0.0832486
[821]	valid_0's binary_logloss: 0.0832497
[822]	valid_0's binary_logloss: 0.08325
[823]	valid_0's binary_logloss: 0.0832487
[824]	valid_0's binary_logloss: 0.0832472

[998]	valid_0's binary_logloss: 0.0831659
[999]	valid_0's binary_logloss: 0.0831661
[1000]	valid_0's binary_logloss: 0.0831661
[1001]	valid_0's binary_logloss: 0.0831687
[1002]	valid_0's binary_logloss: 0.0831698
[1003]	valid_0's binary_logloss: 0.0831693
[1004]	valid_0's binary_logloss: 0.0831682
[1005]	valid_0's binary_logloss: 0.0831693
[1006]	valid_0's binary_logloss: 0.0831689
[1007]	valid_0's binary_logloss: 0.0831685
[1008]	valid_0's binary_logloss: 0.0831663
[1009]	valid_0's binary_logloss: 0.0831663
[1010]	valid_0's binary_logloss: 0.083167
[1011]	valid_0's binary_logloss: 0.0831681
[1012]	valid_0's binary_logloss: 0.0831662
[1013]	valid_0's binary_logloss: 0.083166
[1014]	valid_0's binary_logloss: 0.0831673
[1015]	valid_0's binary_logloss: 0.083165
[1016]	valid_0's binary_logloss: 0.0831662
[1017]	valid_0's binary_logloss: 0.0831661
[1018]	valid_0's binary_logloss: 0.0831674
[1019]	valid_0's binary_logloss: 0.0831672
[1020]	valid_0's binary_logloss: 0.0831689
[1021]	valid_0's

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=64, objective='binary', random_state=None,
       reg_alpha=0.4, reg_lambda=0.0, silent=True, subsample=0.75,
       subsample_for_bin=200000, subsample_freq=1)

In [75]:
print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_ )
y_sub_1 = gbm.predict(X_test, num_iteration=gbm.best_iteration_ )
print(log_loss(y_train,lr.predict_proba(X_train)[:,1]))
print(log_loss(y_val,proba_val))
print()
print(log_loss(y_train,gbm.predict(X_train)))
print(log_loss(y_val,y_pred_1))

print(log_loss(y_val,(y_pred_1 + proba_val)/2))#0.083131236937

Start predicting...


D:\anaconda\lib\site-packages\lightgbm\basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


0.0493250572826
0.0844322874916

0.0764492737721
0.0831649696525
0.083131236937


In [76]:
import xgboost as xgb

model_xgb = xgb.XGBClassifier( n_estimators=2000,random_state =666, nthread = -1,n_jobs=-1)
model_xgb.fit(X_train, y_train,eval_metric='logloss')


y_xpred_1 = model_xgb.predict_proba(X_val)[:,1]
y_xsub_1 = model_xgb.predict_proba(X_test)[:,1]


In [77]:
print(log_loss(y_train,model_xgb.predict_proba(X_train)))
print(log_loss(y_val, y_xpred_1))

0.0818256251833
0.0833334474471


In [78]:
import numpy as np
bes = 1
for i in np.arange(0,1,0.1):
    for f in np.arange(0,1,0.1):
        if(log_loss(y_val,(i * y_pred_1 + f * proba_val + (1 - i - f) * y_xpred_1)) < bes):
            bes = log_loss(y_val,(y_pred_1 + proba_val)/2)
print(bes)

0.083131236937


In [79]:
xx_analyse = pd.DataFrame()
xx_analyse['ture'] = list(y_val)
xx_analyse['pre'] = list(proba_val)
xx_analyse['pre_1'] = list(y_pred_1)
xx_analyse.to_csv('temp.csv',index=False)


sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list(y_sub_1)
sub.to_csv('20180322.txt',sep=" ",index=False)

In [80]:
sub = pd.DataFrame()
sub['instance_id'] = list(test_index)
sub['predicted_score'] = list((proba_sub + y_sub_1)/2)
sub.to_csv('20180323.txt',sep=" ",index=False)

In [81]:
(proba_sub + y_sub_1)/2

array([ 0.00620908,  0.04172497,  0.02587817, ...,  0.01681066,
        0.02186523,  0.01434667])